In [1]:
#ref:- ILDC CJPE GITHUB

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/media/anubhav/New Volume/vidhaan/dataset/data/ILDC_single/ILDC_single.csv")

In [4]:
train_set = df.query(" split=='train' ")
test_set = df.query(" split=='test' ")
validation_set = df.query(" split=='dev' ")

In [5]:
del train_set['split'],train_set['name'],validation_set['split'],validation_set['name']

In [6]:
train_set.head(10)

,text,label
0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1
1,"S. THAKUR, J. Leave granted. These appeals ar...",0
2,"Markandey Katju, J. Leave granted. Heard lear...",1
3,"ALTAMAS KABIR,J. Leave granted. The question ...",1
4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1
5,O R D E R CIVIL APPEAL NOS. 85-86 OF 2002 Thes...,0
6,O R D E R Arising out of S.L.P C No.3358 of ...,1
7,"SANTOSH HEGDE, J. Noticing certain companytra...",0
8,"2004 3 SCR 534 WITH W.P. C No. 852/91, C...",0
9,Arising out SLP c No. 16202 of 2003 With ...,1


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
def preprocess_function(examples):

    return tokenizer(examples["text"], truncation=True)

In [9]:
# t = dict(train_set)

In [10]:
# t

In [11]:
import datasets

In [12]:
train_data = datasets.Dataset.from_pandas(pd.DataFrame(data=train_set))
val_data = datasets.Dataset.from_pandas(pd.DataFrame(data=validation_set))

In [13]:
tokenized_train_set = train_data.map(preprocess_function, batched=True)
tokenized_val_set = val_data.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f19d1aa1830> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [15]:
tf_train_dataset = tokenized_train_set.to_tf_dataset(

    columns=["text"],

    label_cols=["label"],

    shuffle=True,

    collate_fn=data_collator,

    batch_size=8,

)

tf_validation_dataset = tokenized_val_set.to_tf_dataset(

    columns=["text"],

    label_cols=["label"],

    shuffle=False,

    collate_fn=data_collator,

    batch_size=8,

)

In [16]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3


ValueError: in user code:

    File "/home/anubhav/miniconda3/envs/vidhaan/lib/python3.7/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/anubhav/miniconda3/envs/vidhaan/lib/python3.7/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/anubhav/miniconda3/envs/vidhaan/lib/python3.7/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/anubhav/miniconda3/envs/vidhaan/lib/python3.7/site-packages/transformers/modeling_tf_utils.py", line 1398, in train_step
        y_pred = self(x, training=True)
    File "/home/anubhav/miniconda3/envs/vidhaan/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filee535u1i3